In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import missingno
import ipywidgets as widgets
from IPython.display import display
import joblib
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


In [ ]:
barcelona = pd.read_csv('/content/barcelona_clean_zip_reduce.csv', sep = ";")
barcelona

,zip_mean_price,price_m2_mean_log,log_price,floor,propertyType,size,exterior,rooms,bathrooms,hasLift,hasParkingSpace,hasSwimmingPool,hasTerrace,hasAirConditioning,hasBoxRoom,hasGarden,zip_code
0,14.060566,8.978610,14.285514,1,flat,114.0,False,3,2,True,False,No data,True,True,No data,No data,8010
1,13.807605,9.000411,14.272935,1,flat,202.0,True,3,2,False,False,No data,No data,No data,No data,No data,8008
2,14.237496,9.169621,13.795308,4,penthouse,156.0,True,2,2,True,False,No data,True,No data,No data,No data,8007
3,13.185494,8.660511,13.676248,1,flat,129.0,True,1,2,True,False,No data,No data,No data,No data,No data,8003
4,13.185494,8.660511,13.112313,3,flat,91.0,True,1,1,True,False,No data,True,No data,No data,No data,8003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8913,14.093444,8.850762,15.424948,7,duplex,500.0,True,6,5,True,False,True,True,True,True,True,8034
8914,14.093444,8.850762,15.538277,No data,chalet,1000.0,False,11,6,False,False,No data,True,False,True,True,8034
8915,14.305278,8.643760,15.590463,No data,chalet,560.0,False,5,6,False,False,True,True,True,True,True,8017
8916,14.305278,8.643760,15.823725,No data,chalet,900.0,False,7,4,False,False,True,False,False,False,False,8017


In [ ]:
barcelona.columns

Index(['zip_mean_price', 'log_price', 'floor', 'propertyType', 'size',
       'exterior', 'rooms', 'bathrooms', 'hasLift', 'hasParkingSpace',
       'hasSwimmingPool', 'hasTerrace', 'hasAirConditioning', 'hasBoxRoom',
       'hasGarden', 'zip_code'],
      dtype='object')

In [ ]:

X = barcelona.drop(columns=['log_price'])  
y = barcelona['log_price'] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [ ]:
barcelona.columns

Index(['zip_mean_price', 'log_price', 'floor', 'propertyType', 'size',
       'exterior', 'rooms', 'bathrooms', 'hasLift', 'hasParkingSpace',
       'hasSwimmingPool', 'hasTerrace', 'hasAirConditioning', 'hasBoxRoom',
       'hasGarden', 'zip_code'],
      dtype='object')

In [ ]:
best_params = {
    'n_estimators': 300,
    'max_depth': 10,
    'learning_rate': 0.05,
    'subsample': 0.8,
    'colsample_bytree': 1.0
}

xgboost_model = xgb.XGBRegressor(
    n_estimators=best_params['n_estimators'],
    max_depth=best_params['max_depth'],
    learning_rate=best_params['learning_rate'],
    subsample=best_params['subsample'],
    colsample_bytree=best_params['colsample_bytree'],
    random_state=42
)

xgboost_model.fit(X_train, y_train)

y_pred = xgboost_model.predict(X_test)

rmse = mean_squared_error(y_test, y_pred) ** 0.5
r2 = r2_score(y_test, y_pred)

print("XGBoost with optimized hyperparameters:")
print(f"  RMSE: {rmse:.4f}")
print(f"  R²: {r2:.4f}")

XGBoost with optimized hyperparameters:
  RMSE: 0.2686
  R²: 0.9238


In [ ]:
xgb_model_prova_1 = joblib.load('/TFM Mercado Inmobiliario - Kschool/Metodología de Modelado/9. Código y datasets/Barcelona/Codigo/xgboost_model.pkl')

In [ ]:
features = [
    'zip_mean_price', 'floor', 'propertyType', 'size', 'exterior', 'rooms',
       'bathrooms', 'hasLift', 'hasParkingSpace', 'hasSwimmingPool',
       'hasTerrace', 'hasAirConditioning', 'hasBoxRoom', 'hasGarden',
       'zip_code']

In [ ]:
barcelona = barcelona[features]

In [ ]:
prova_1 = xgb_model_prova_1.get_booster().feature_names

In [ ]:
barcelona = pd.get_dummies(barcelona, drop_first=True)

predicciones = xgb_model_prova_1.predict(barcelona)

In [ ]:
predicciones = pd.DataFrame(predicciones, columns=['log_price'])
predicciones



,log_price
0,13.66
1,14.25
2,13.93
3,13.70
4,13.11
...,...
8913,15.42
8914,15.53
8915,15.33
8916,15.24


In [ ]:
predicciones['price_pred'] = np.exp(predicciones['log_price'])

In [ ]:
pd.options.display.float_format = '{:.2f}'.format

In [ ]:
predicciones

,log_price,price_pred
0,13.66,854955.75
1,14.25,1536488.88
2,13.93,1119030.00
3,13.70,889093.75
4,13.11,492755.38
...,...,...
8913,15.42,4970036.00
8914,15.53,5551889.00
8915,15.33,4563450.00
8916,15.24,4135669.50


In [ ]:
barcelona = pd.read_csv('/TFM Mercado Inmobiliario - Kschool/Metodología de Modelado/9. Código y datasets/Barcelona/Datasets/result_barcelona_districs_zip_final.csv', sep = ";")
barcelona

,propertyCode,thumbnail,externalReference,numPhotos,floor,price,propertyType,operation,size,exterior,...,hasSwimmingPool,hasTerrace,hasAirConditioning,hasBoxRoom,hasGarden,zip_code,size_category,cleaned_description,cinema,distancia_cinema
0,99117414,https://img4.idealista.com/blur/WEB_LISTING-M/...,BCNP2854,22,1,1600000.00,flat,sale,114.00,False,...,No data,True,True,No data,No data,8010,Very Large,impressive apartment sale 163m2 terrace locate...,Cinema 5D,5819924.17
1,101300670,https://img4.idealista.com/blur/WEB_LISTING-M/...,BCNP5151,29,1,1580000.00,flat,sale,202.00,True,...,No data,No data,No data,No data,No data,8008,Extra Large,sothebys international realty pleased present ...,Cinema 5D,5820797.14
2,104772587,https://img4.idealista.com/blur/WEB_LISTING-M/...,BCNP2636,45,4,980000.00,penthouse,sale,156.00,True,...,No data,True,No data,No data,No data,8007,Extra Large,four luxurious boutique apartment eixample spe...,Cinema 5D,5820229.13
3,100770574,https://img4.idealista.com/blur/WEB_LISTING-M/...,BCNP5175,23,1,870000.00,flat,sale,129.00,True,...,No data,No data,No data,No data,No data,8003,Very Large,exclusive property offer authentic characteris...,Cinema 5D,5818938.80
4,105932101,https://img4.idealista.com/blur/WEB_LISTING-M/...,BCN53025,35,3,495000.00,flat,sale,91.00,True,...,No data,True,No data,No data,No data,8003,Large,located emblematic born neighbourhood barcelon...,Cinema 5D,5819331.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8913,105882581,https://img4.idealista.com/blur/WEB_LISTING-M/...,No data,2,7,5000000.00,duplex,sale,500.00,True,...,True,True,True,True,True,8034,Extra Large,magnificent penthouse completely renovated sal...,Cinema 5D,5823345.86
8914,81411842,https://img4.idealista.com/blur/WEB_LISTING-M/...,CSM 163085V,29,No data,5600000.00,chalet,sale,1000.00,False,...,No data,True,False,True,True,8034,Extra Large,elegant mansion privileged location upper area...,Cinema 5D,5823887.90
8915,103461663,https://img4.idealista.com/blur/WEB_LISTING-M/...,Casa Claver,36,No data,5900000.00,chalet,sale,560.00,False,...,True,True,True,True,True,8017,Extra Large,one home present casa claver unique opportunit...,Cinema 5D,5824448.65
8916,98964029,https://img4.idealista.com/blur/WEB_LISTING-M/...,B07442BA,30,No data,7450000.00,chalet,sale,900.00,False,...,True,False,False,False,False,8017,Extra Large,spectacular modern house located upper area ba...,Cinema 5D,5824809.57


In [ ]:
barcelona = pd.concat([barcelona, predicciones], axis = 1)

In [ ]:
barcelona = barcelona[['price', 'price_pred', 'propertyCode']]
barcelona

,price,price_pred,propertyCode
0,1600000.00,854955.75,99117414
1,1580000.00,1536488.88,101300670
2,980000.00,1119030.00,104772587
3,870000.00,889093.75,100770574
4,495000.00,492755.38,105932101
...,...,...,...
8913,5000000.00,4970036.00,105882581
8914,5600000.00,5551889.00,81411842
8915,5900000.00,4563450.00,103461663
8916,7450000.00,4135669.50,98964029


In [ ]:
barcelona['desviations'] = barcelona['price'] - barcelona['price_pred']
barcelona

,price,price_pred,propertyCode,desviations
0,1600000.00,854955.75,99117414,745044.25
1,1580000.00,1536488.88,101300670,43511.12
2,980000.00,1119030.00,104772587,-139030.00
3,870000.00,889093.75,100770574,-19093.75
4,495000.00,492755.38,105932101,2244.62
...,...,...,...,...
8913,5000000.00,4970036.00,105882581,29964.00
8914,5600000.00,5551889.00,81411842,48111.00
8915,5900000.00,4563450.00,103461663,1336550.00
8916,7450000.00,4135669.50,98964029,3314330.50


In [ ]:
# barcelona['desviations'] = barcelona['desviations'].abs()
# barcelona

| Statistic | price_pred   | deviations   |
|-----------|--------------|--------------|
| count     | 8918.00      | 8918.00      |
| mean      | 562563.94    | 50611.32     |
| std       | 892947.75    | 200622.93    |
| min       | 30905.16     | 7.97         |
| 25%       | 166542.34    | 5756.62      |
| 50%       | 309827.84    | 15494.48     |
| 75%       | 559799.86    | 39641.44     |
| max       | 20039908.00  | 7760492.00   |


In [ ]:
barcelona[['price_pred','desviations']].describe()

,price_pred,desviations
count,8918.00,8918.00
mean,564365.50,10092.13
std,903217.25,202347.84
min,28308.85,-7079554.00
25%,166336.06,-10000.36
50%,311411.06,282.38
75%,564654.45,13732.67
max,20669064.00,7782493.00


In [ ]:
barcelona.describe()

,price,price_pred,propertyCode,desviations
count,8918.00,8918.00,8918.00,8918.00
mean,574457.67,564365.50,103850500.39,10092.13
std,949811.30,903217.25,5824999.45,202347.84
min,12000.00,28308.85,320294.00,-7079554.00
25%,166500.00,166336.06,103955183.75,-10000.36
50%,310000.00,311411.06,105088150.00,282.38
75%,589750.00,564654.45,105655310.50,13732.67
max,21500000.00,20669064.00,106053114.00,7782493.00


In [ ]:
primer = 5000
segundo = 15000
tercer = 30000
cuarto = 45000
quinto = 100000


bins = [-np.inf, -quinto, -cuarto, -tercer, -segundo, -primer, 0, primer, segundo, tercer, cuarto, quinto, np.inf]

labels = ['1_Extrema_neg', '2_Muy alta_neg', '3_Alta_neg', '4_Media_neg', '5_Baja_neg', '6_Muy baja_neg',
          '7_Muy baja_pos', '8_Baja_pos', '9_Media_pos', '10_Alta_pos', '11_Muy alta_pos',
          '12_Extrema_pos']  

barcelona['deviacion_class'] = pd.cut(barcelona['desviations'],
                                       bins=bins,
                                       labels=labels,
                                       right=False)


barcelona['deviacion_class'].value_counts().sort_index()

deviacion_class
1_Extrema_neg       258
2_Muy alta_neg      441
3_Alta_neg          291
4_Media_neg         775
5_Baja_neg         1249
6_Muy baja_neg     1359
7_Muy baja_pos     1163
8_Baja_pos         1259
9_Media_pos         774
10_Alta_pos         378
11_Muy alta_pos     505
12_Extrema_pos      466
Name: count, dtype: int64

In [ ]:
barcelona

,price,price_pred,propertyCode,desviations,deviacion_class
0,1600000.00,854955.75,99117414,745044.25,12_Extrema_pos
1,1580000.00,1536488.88,101300670,43511.12,10_Alta_pos
2,980000.00,1119030.00,104772587,-139030.00,1_Extrema_neg
3,870000.00,889093.75,100770574,-19093.75,4_Media_neg
4,495000.00,492755.38,105932101,2244.62,7_Muy baja_pos
...,...,...,...,...,...
8913,5000000.00,4970036.00,105882581,29964.00,9_Media_pos
8914,5600000.00,5551889.00,81411842,48111.00,11_Muy alta_pos
8915,5900000.00,4563450.00,103461663,1336550.00,12_Extrema_pos
8916,7450000.00,4135669.50,98964029,3314330.50,12_Extrema_pos


In [ ]:
barcelona.to_csv('/TFM Mercado Inmobiliario - Kschool/Metodología de Modelado/9. Código y datasets/Barcelona/Datasets/result_barcelona_price_predict.csv', sep = ";", index = False)